# Notebook 1 – Data Cleaning, Feature Engineering, & Entity Resolution
**Project:** Judicial Vacancy → Nomination/Confirmation Pipeline

*Initial draft generated via ChatGPT model o3 on 2025-07-12T02:40:38.399372Z*

In [ ]:

import json
import os
import re
import string
import sys
from datetime import datetime
from pathlib import Path

import pandas as pd
from loguru import logger
from rapidfuzz import fuzz, process

# Add the project root to the path so we can import our modules
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from nomination_predictor.config import INTERIM_DATA_DIR, RAW_DATA_DIR

# Setup logging
logger.remove()  # Remove default handler
logger.add(sys.stderr, format="<green>{time:YYYY-MM-DD HH:mm:ss}</green> | <level>{level}</level> | <cyan>{function}</cyan> - <level>{message}</level>", level="INFO")

7

In [ ]:

# ---------------------------------------------------------------------------
# Helper: clean / normalize names
def clean_name(name: str) -> str:
    if pd.isna(name):
        return ""
    name = str(name).upper()
    name = re.sub(r"[\.,]", "", name)          # drop punctuation
    name = re.sub(r"\s+", " ", name).strip()
    return name

def split_name(name: str):
    """
    Very naive splitter: returns first, middle (maybe empty), last
    """
    parts = clean_name(name).split()
    if not parts:
        return "", "", ""
    if len(parts) == 1:
        return parts[0], "", ""
    if len(parts) == 2:
        return parts[0], "", parts[1]
    return parts[0], " ".join(parts[1:-1]), parts[-1]

def normalised_court(text: str) -> str:
    if pd.isna(text):
        return ""
    text = text.upper().replace("UNITED STATES", "").replace("U.S.", "").strip()
    text = re.sub(r"\s+", " ", text)
    return text

In [ ]:

# Expected input files (produced by Notebook 0)
fjc_judges    = pd.read_csv(RAW_DATA_DIR / "judges.csv")
fjc_service   = pd.read_csv(RAW_DATA_DIR / "federal_judicial_service.csv")
cong_nominees = pd.read_csv(RAW_DATA_DIR / "congress_nominees_cache.csv")
cong_noms     = pd.read_csv(RAW_DATA_DIR / "congress_nominations_cache.csv")

print("Loaded:",
      len(fjc_judges), "judges",
      len(fjc_service), "service records",
      len(cong_nominees), "congress nominees",
      len(cong_noms), "nominations")

Loaded: 4022 judges 4720 service records 285 congress nominees 285 nominations


In [14]:

# --- Clean Congress nominees ------------------------------------------------
cong_nominees["full_name_clean"] = cong_nominees["full_name"].apply(clean_name)
cong_nominees[["first","middle","last"]] = cong_nominees["full_name_clean"].apply(
    lambda n: pd.Series(split_name(n)))

cong_nominees["court_clean"] = cong_nominees["organization"].apply(normalised_court)
cong_nominees["nomination_date"] = pd.to_datetime(cong_nominees["nomination_date"])

# --- Clean FJC judges -------------------------------------------------------
fjc_judges["full_name_clean"] = fjc_judges["name_full"].apply(clean_name)
fjc_judges[["first","middle","last"]] = fjc_judges["full_name_clean"].apply(
    lambda n: pd.Series(split_name(n)))

# We'll need a mapping from nid to service records for date & court validation
fjc_service["court_clean"] = fjc_service["court"].apply(normalised_court)
fjc_service["nomination_date"] = pd.to_datetime(fjc_service["nomination_date"], errors="coerce")
fjc_service["commission_date"] = pd.to_datetime(fjc_service["commission_date"], errors="coerce")

KeyError: 'full_name'

In [ ]:

# Block by last name exact match
blocks = {}
for lname, group in fjc_judges.groupby("last"):
    blocks[lname] = group

def candidate_fjc_rows(row):
    return blocks.get(row["last"], pd.DataFrame())

In [ ]:

def best_match(row):
    candidates = candidate_fjc_rows(row)
    if candidates.empty:
        return pd.NA, 0.0
    # Compute combined score: name similarity + court similarity + date proximity
    best_score = 0.0
    best_nid = pd.NA
    for _, cand in candidates.iterrows():
        name_score = fuzz.token_set_ratio(row["full_name_clean"], cand["full_name_clean"])
        # Use service records to find any matching nomination date
        entries = fjc_service[fjc_service["nid"] == cand["nid"]]
        date_score = 0
        court_score = 0
        if not entries.empty:
            # Smallest absolute diff in days
            diffs = (entries["nomination_date"] - row["nomination_date"]).abs().dt.days
            date_score = 100 - diffs.min() if diffs.notna().any() else 0
            # any court string overlap
            if row["court_clean"]:
                if any(row["court_clean"] in c for c in entries["court_clean"]):
                    court_score = 100
                else:
                    court_score = max(fuzz.partial_ratio(row["court_clean"], c) for c in entries["court_clean"])
        total = 0.6*name_score + 0.3*date_score + 0.1*court_score
        if total > best_score:
            best_score, best_nid = total, cand["nid"]
    return best_nid, round(best_score,1)

cong_nominees[["match_nid","match_score"]] = cong_nominees.apply(
    best_match, axis=1, result_type="expand")

In [ ]:

THRESHOLD = 80
matches = cong_nominees[cong_nominees["match_score"] >= THRESHOLD].copy()
print(f"Matched {len(matches)}/{len(cong_nominees)} nominees with score ≥ {THRESHOLD}")
matches.to_csv(INTERIM_DATA_DIR / "congress_fjc_nominee_matches.csv", index=False)

# Save the cleaned interim datasets for downstream notebooks
cong_nominees.to_csv(INTERIM_DATA_DIR / "congress_nominees_cleaned.csv", index=False)
fjc_judges.to_csv(INTERIM_DATA_DIR / "fjc_judges_cleaned.csv", index=False)
fjc_service.to_csv(INTERIM_DATA_DIR / "fjc_service_cleaned.csv", index=False)